In [15]:
import pycurl
from io import BytesIO
import bs4
from bs4 import BeautifulSoup

import pandas as pd

In [16]:
def search(url):
    buffer = BytesIO()
    storage = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()
    body = buffer.getvalue()
    soup = BeautifulSoup(body, 'html.parser')
    return soup

def get_author_id(term):
    # create empty df
    result = pd.DataFrame(columns=['name', 'sinta_id', 'affiliation', 'NIDN', 'expertise'])
    
    url = 'https://sinta.ristekbrin.go.id/authors?q='+term.replace(' ','+')+'&view=&search=1&ag='
    
    # look only at science
    #url = 'https://sinta.ristekbrin.go.id/authors?q='+term.replace(' ','+')+'&view=science&view=&search=1&ag='
    
    author_search = search(url)
    
    for num, i in enumerate(author_search.find_all('a', {"class":"text-blue"})): #get first item
        #get name and sinta id
        result.loc[num, 'name'] = i.contents[0]
        result.loc[num, 'sinta_id'] =  i.attrs['href'].replace('/authors/detail/?id=','').replace('&view=overview','')
        # get affiliation and NIDN
        for x in i.find_next('dd'):
            result.loc[num, 'affiliation'] = x.contents[0]
            result.loc[num, 'NIDN'] = x.find_next('dd').contents[2].replace(' : ', '')
        # Get expertise
        expertise = [] # empty container
        expertise1 = i.find_next('a', {'class':"area-item-small"}) # get first expertise
        ctr = 0 # helper counter
        while ctr < 3 and isinstance(expertise1, bs4.element.Tag): # loop until NoneType & max 3 expertise
            expertise.append(expertise1.contents[0])
            expertise1 = expertise1.find_next('a') # get next expertise
            if expertise1['class'][0] == 'area-item-small': # if expertise feature, continue
                ctr = ctr + 1
            else: # if not expertise feature pass
                ctr = 3     
        result.loc[num, 'expertise'] = expertise
    return result

In [25]:
data = get_author_id('indra lesmana')
data

,name,sinta_id,affiliation,NIDN,expertise
0,INDRA LESMANA,6081151,Universitas Riau,0008038406,"[Aquaculture, Fish Genetics and Reproduction B..."
1,INDRA LESMANA,6023357,Universitas Gadjah Mada,1120170054,[Molecular Genetics]
2,SYAHMIRZA INDRA LESMANA,5989126,Universitas Esa Unggul,0307076801,"[Sport Physiotherapy, Sport Sciences]"


In [22]:
import sinta_scraper as sinta

In [24]:
author = sinta.author(data.loc[1, 'sinta_id'])
author

{'id': '6023357',
 'name': 'INDRA LESMANA',
 'url': 'http://sinta.ristekbrin.go.id/authors/detail?id=6023357&view=overview',
 'affiliation': {'id': '384',
  'name': 'Universitas Gadjah Mada',
  'url': 'http://sinta.ristekbrin.go.id/affiliations/detail/?id=384&view=overview'},
 'department': 'Biologi Tropika',
 'areas': ['Molecular Genetics'],
 'score': {'overall': 2.44,
  '3_years': 1.61,
  'overall_v2': 229.0,
  '3_years_v2': 124.0},
 'rank': {'national': 31691,
  '3_years_national': 31533,
  'affiliation': 1317,
  '3_years_affiliation': 1101},
 'scopus': {'documents': 7,
  'citations': 2,
  'h-index': 1,
  'i10-index': 0,
  'g-index': 1,
  'articles': 4,
  'conferences': 3,
  'others': 0,
  'Q1': 1,
  'Q2': 0,
  'Q3': 2,
  'Q4': 4,
  'undefined': 0},
 'scholar': {'documents': 4,
  'citations': 1,
  'h-index': 1,
  'i10-index': 0,
  'g-index': 1},
 'wos': {'documents': 0,
  'citations': None,
  'h-index': None,
  'i10-index': None,
  'g-index': None},
 'sinta': {'S0': 0,
  'S1': 0,
  

In [21]:
! git mv 'result UGM_2.csv' 'figures/result UGM_2.csv'